In [14]:
import keras
import tensorflow as tf 
import seaborn as sns
import cv2
import numpy as np 
import pandas as pd
from models import matting_net
from data_generator import *
from metrics import * 

In [7]:
# model path for using in EDA
MODEL_PATH = "./trained_models/Strongmodels/02-0.09.h5"
IMG_PATHS = "./dataset/img_paths_with_supervisely_nosmallobject.npy"

In [8]:
img_paths = np.load(IMG_PATHS)
img_paths[:2]

array(['./dataset/selfie/testing/00001.png',
       './dataset/selfie/testing/00002.png'], dtype='<U135')

In [5]:
# test and store ious
model = matting_net((256,256,3), train=False)
model.load_weights(MODEL_PATH)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 8)  224         input_1[0][0]                    
__________________________________________________________________________________________________
re_lu_1 (ReLU)                  (None, 256, 256, 8)  0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
separable_conv2d_1 (SeparableCo (None, 256, 256, 8)  144         re_lu_1[0][0]                    
____________________________________________________________________________________________

In [9]:
# data feeding logic
test_params = {
            'dim': (256,256),
            'batch_size': 32,
            'n_channels': 3,
            'shuffle': False,
            'augment': False,
        }

gen = DataGeneratorMatting(img_paths, **test_params)

In [17]:
model.compile(loss=ce_dice_focal_combined_loss,
                   optimizer=keras.optimizers.Adam(),
                   metrics=[iou_coef])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [18]:
history = model.evaluate_generator(gen, verbose=1)

ValueError: Error when checking model target: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 2 arrays: [array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
    ...